In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy import signal, linalg

In [8]:
def detrend(data, axis=-1, type='linear', bp=0):
    """
    Remove linear trend along axis from data. (Simplified from Scipy)
    """
    if type not in ['linear', 'l', 'constant', 'c']:
        raise ValueError("Trend type must be 'linear' or 'constant'.")  

    data = np.asarray(data)
    if type in ['constant', 'c']:
        data_detrended = data - np.mean(data, axis, keepdims=True)
        return data_detrended
    else:
        dshape = data.shape
        N = dshape[axis]
        bp = np.sort(np.unique(np.r_[0, bp, N]))
        if np.any(bp > N):
            raise ValueError("Breakpoints must be less than length "
                             "of data along given axis.")
        Nreg = len(bp) - 1
        
        # Restructure data so that axis is along first dimension and
        # all other dimensions are collapsed into second dimension
        rnk = len(dshape)
        if axis < 0:
            axis = axis + rnk
        newdims = np.r_[axis, 0:axis, axis+1:rnk]
        newdata = np.reshape(np.transpose(data, tuple(newdims)),
                            (N, np.prod(dshape) // N))
        newdata = newdata.copy() # make sure we have a copy

        # Find leastsq fit and remove it for each piece
        for m in range(Nreg):
            Npts = bp[m + 1] - bp[m]
            A = np.ones((Npts, 2))
            A[:, 0] = np.arange(1, Npts+1) / Npts
            sl = slice(bp[m], bp[m+1])
            coef, _, _, _ = linalg.lstsq(A, newdata[sl])
            newdata[sl] = newdata[sl] - np.dot(A, coef)
        
        # Put data back in original shape
        tdshape = np.take(dshape, newdims, 0)
        data_detrended = np.reshape(newdata, tuple(tdshape))
        vals = list(range(1, rnk))
        olddims = vals[:axis] + [0] + vals[axis:]
        data_detrended = np.transpose(data_detrended, tuple(olddims))
        
        return data_detrended

In [3]:
rng = default_rng()
npoints = 1000
noise = rng.standard_normal(npoints)
x = 3 + 2 * np.linspace(0, 1, npoints) + noise

In [4]:
(signal.detrend(x) - noise).max()

-0.0016516865224826538

In [9]:
(detrend(x) - noise).max()

-0.0016516865224826538

In [10]:
plt.figure()
plt.plot(x, label='Shifted')
plt.plot(signal.detrend(x), label='Detrend:Scipy')
plt.plot(detrend(x), label='Detrend:Mine')
plt.legend()
plt.show()

qt.qpa.drawing: Layer-backing can not be explicitly controlled on 10.14 when built against the 10.14 SDK
